In [17]:
%%cu
#include <bits/stdc++.h>
using namespace std;

// Kernel function for matrix multiplication on GPU
__global__
void GPUmatmul(int N, int *x, int  *y, int *ans)
{
	//calculates unique thread ID in the block
	int t= (blockDim.x*blockDim.y)*threadIdx.z+(threadIdx.y*blockDim.x)+(threadIdx.x);
 
	//calculates unique block ID in the grid
	int b= (gridDim.x*gridDim.y)*blockIdx.z+(blockIdx.y*gridDim.x)+(blockIdx.x);

	//block size (this is redundant though)
	int T= blockDim.x*blockDim.y*blockDim.z;

	//grid size (this is redundant though)
	int B= gridDim.x*gridDim.y*gridDim.z;
	
	/*
	  Each cell in the matrix is assigned to a different thread. 
	  Each thread do O(N*number of asssigned cell) computation.
	  Assigned cells of different threads does not overlape with
	  each other. And so no need for synchronization.
	 */
	 
    for (int i=b;i<N;i+=B)
    {
		for(int j=t;j<N;j+=T)
		{
			for(int k=0;k<N;k++)
			{
				ans[i*N+j]+=(x[i*N+k]*y[k*N+j]);
			}
		}
	}
}

//simple matrix multiplication
void CPUmatmul(int N,int *x, int *y, int *ans)
{
	for(int i=0;i<N;i++)
	{
		for(int j=0;j<N;j++)
		{
			for(int k=0;k<N;k++)
			{
				ans[i*N+j]+=(x[i*N+k]*y[k*N+j]);
			}
		}
	}
}


int main(void)
{
	//size of matrix
	int N = 64;
	
	int *x, *y, *ans;

	// Allocate Unified Memory – accessible from CPU or GPU
	cudaMallocManaged(&x, N*N*sizeof(int));
	cudaMallocManaged(&y, N*N*sizeof(int));
	cudaMallocManaged(&ans, N*N*sizeof(int));

	// initialize x,y and ans arrays on the host
	for (int i = 0; i < N; i++) 
	{
		for(int j=0;j<N;j++)
		{
			x[i*N+j]=8;
			y[i*N+j]=(i==j?1:0);
			ans[i*N+j]=0;
		}
	}
   
	clock_t t;
	double avg=0;
	//taking average  of four computations 
	cout<<"\n ----Start CPU computation----"<<endl;
	for(int i=0;i<=3;i++)
	{
		t=clock();
		CPUmatmul(N, x, y,ans);
		t = clock() - t;
		if(i)avg+=t;  //we will ignore the first run
		cout<<"It took CPU-"<<i<<" "<<((double)t/CLOCKS_PER_SEC)*1000<<" ms.\n";
	}
	avg/=3;
	avg/=CLOCKS_PER_SEC;
	avg*=1000;
	cout<<"\nIt took "<< avg<<" ms on avg using CPU.\n";
	cout<<"\n ----End of CPU Computation----";

	// initialize x,y and ans arrays on the host
	for (int i = 0; i < N; i++) 
	{
		for(int j=0;j<N;j++)
		{
			x[i*N+j]=8;
			y[i*N+j]=(i==j?1:0);
			ans[i*N+j]=0;
		}
	}

	//taking average  of four computations 
	avg=0;
	cout<<"\n\n\n----Starting GPU computation----"<<endl;
	// Run kernel on GPU
	for(int i=0;i<=3;i++)
	{
		t=clock();
		GPUmatmul<<<dim3(16,16,16), dim3(16,8,8)>>>(N, x, y,ans);
		cudaDeviceSynchronize();
		t = clock() - t;
		if(i)avg+=t; //we will ignore the first run
    cout<<"It took GPU-"<<i<<" "<<((double)t/CLOCKS_PER_SEC)*1000<<" ms.\n";
	}
	avg/=3;
	avg/=CLOCKS_PER_SEC;
	avg*=1000;
	cout<<"\n It took "<<avg<<" ms on avg using GPU.\n";
	cout<<"\n ----End of GPU Computation----";
  
	// Free memory
	cudaFree(x);
	cudaFree(y); 
  return 0;
}


 ----Start CPU computation----
It took CPU-0 1.098 ms.
It took CPU-1 1.25 ms.
It took CPU-2 1.125 ms.
It took CPU-3 1.123 ms.

It took 1.166 ms on avg using CPU.

 ----End of CPU Computation----


----Starting GPU computation----
It took GPU-0 0.532 ms.
It took GPU-1 0.055 ms.
It took GPU-2 0.053 ms.
It took GPU-3 0.045 ms.

 It took 0.051 ms on avg using GPU.

 ----End of GPU Computation----
